# <p style="padding:10px;background-color:#85BB65;margin:0;color:white;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Inconsistency</p>


Inconsistency occurs when two values or more in the data set contradict each other.

In [1]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer


In [2]:
df = pd.read_csv("car_price_1.csv")
df.head()

,carbody,drivewheel,enginelocation,fueltype,aspiration,doornumber,cylindernumber,enginetype,fuelsystem,symboling,...,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company name
0,convertible,rwd,front,gas,std,two,four,dohc,mpfi,3,...,130,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,convertible,rwd,front,gas,std,two,four,dohc,mpfi,3,...,130,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,hatchback,rwd,front,gas,std,two,six,ohcv,mpfi,1,...,152,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,sedan,fwd,front,gas,std,four,four,ohc,mpfi,2,...,109,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,sedan,4wd,front,gas,std,four,five,ohc,mpfi,2,...,136,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


**Say we're interested in cleaning up the "company name" column to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them and also there's a more efficient way to do this, though!**

# <b>I <span style='color:#85BB65'>|</span> fix it manually:</b> 


In [3]:
df1=df.copy()

In [4]:
print(f"number of unique values = {df1['company name'].nunique()} \n")

print(df1['company name'].unique())

number of unique values = 28 

['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'maxda' 'mazda' 'buick' 'mercury' 'mitsubishi' 'Nissan' 'nissan'
 'peugeot' 'plymouth' 'porsche' 'porcshce' 'renault' 'saab' 'subaru'
 'toyota' 'toyouta' 'vokswagen' 'volkswagen' 'vw' 'volvo']


#### **<mark style="background-color:#85BB65;color:white;border-radius:5px;opacity:1.0">Note that</mark>**

* **maxda = mazda**
* **Nissan = nissan**
* **porsche = porcshce**
* **toyota = toyouta**
* **vokswagen = volkswagen = vw**

In [5]:
df1.replace({'company name':{"maxda":"mazda" , 
                        "Nissan":"nissan" ,
                        "porcshce":"porsche" ,
                        "toyouta":"toyota" ,
                        "vokswagen":"volkswagen" ,
                        "vw":"volkswagen"}},inplace=True)

In [6]:
print(f"number of unique values = {df1['company name'].nunique()} \n")

print(df1['company name'].unique())

number of unique values = 22 

['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'buick' 'mercury' 'mitsubishi' 'nissan' 'peugeot' 'plymouth'
 'porsche' 'renault' 'saab' 'subaru' 'toyota' 'volkswagen' 'volvo']


now we overcome the inconsistencies problem manually but this dataset is small enough that we could probably could correct errors by hand, but that approach doesn't scale well. (Would you want to correct a thousand errors by hand? What about ten thousand? Automating things as early as possible is generally a good idea. Plus, it’s fun!

# <b>II <span style='color:#85BB65'>|</span> fuzzy matching:</b> 

**<span style='color:#85BB65'>Fuzzy matching</span>**: **The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (replace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.**

In [7]:
df2=df.copy()

first thing I'm going to do is make everything lower case and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.

In [8]:
# convert to lower case
df2['company name'] = df2['company name'].str.lower()
# remove trailing white spaces
df2['company name'] = df2['company name'].str.strip()

In [9]:
print(f"number of unique values = {df2['company name'].nunique()} \n")

companies_name = df2['company name'].unique()
companies_name

number of unique values = 27 



array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

**<span style='color:#85BB65'>fuzzywuzzy library </span>**
 **returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to any string.**

In [10]:
# get the top 10 closest matches to "toyota"
matches = fuzzywuzzy.process.extract("toyota", companies_name , limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('toyota', 100),
 ('toyouta', 92),
 ('plymouth', 43),
 ('honda', 36),
 ('volvo', 36),
 ('peugeot', 31),
 ('chevrolet', 27),
 ('vokswagen', 27),
 ('volkswagen', 25),
 ('alfa-romero', 24)]

**We can see that two of the items in the cities are very close to "toyota": "toyota" and "toyouta". Let's replace all rows in our "company name" column that have a ratio of > 90 with "toyota". So, let's  write a function to do that.**

In [11]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 80):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [12]:
# use the function we just wrote to replace close matches to "toyota" with "toyota"
replace_matches_in_column(df=df2, column='company name', string_to_match="toyota" , min_ratio = 90)

All done!


In [13]:
print(f"number of unique values = {df2['company name'].nunique()} \n")

print(df2['company name'].unique())

number of unique values = 26 

['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'maxda' 'mazda' 'buick' 'mercury' 'mitsubishi' 'nissan' 'peugeot'
 'plymouth' 'porsche' 'porcshce' 'renault' 'saab' 'subaru' 'toyota'
 'vokswagen' 'volkswagen' 'vw' 'volvo']


**Now we only have "toyota" in our dataframe . you can change others with function and you didn't have to change anything by hand.**

***

<br>

<div style="text-align: center;">
   <span style="font-size: 4.5em; font-weight: bold; font-family: Arial;">THANK YOU!</span>
</div>/

<br>
<br>

<div style="text-align: center;">
    <span style="font-size: 5em;">✔️</span>
</div>

<br>

<div style="text-align: center;">
   <span style="font-size: 1.4em; font-weight: bold; font-family: Arial; max-width:1200px; display: inline-block;">
       If you find this notebook useful, I'd greatly appreciate your upvote!
   </span>
</div>
